In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import random
from dataclasses import dataclass

@dataclass
class TrainCfg:
    batch_size: int = 128
    max_epochs: int = 150
    lr: float = 3e-4
    device: str = "cuda" if torch.cuda.is_available() else "cpu"


CFG = TrainCfg()

In [3]:
class CardioRiskNet(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        return torch.sigmoid(self.fc3(x))


In [ ]:
import torch
from torch.utils.data import Dataset
import random
from config import CFG

class SyntheticPatientDataset(Dataset):
    def __init__(self, num_samples=2000, cfg=CFG):
        self.features = []
        self.targets = []

        for _ in range(num_samples):
            # Core features
            ecg_prob = random.uniform(0, 1)
            exercise = random.uniform(0, 1)
            diet = random.uniform(0, 1)
            sleep = random.uniform(0, 1)
            smoking = random.choice([0, 1])
            alcohol = random.choice([0, 1])
            age = random.uniform(20, 80)  # raw age
            bmi = random.uniform(18.5, 30)
            bp = random.uniform(90, 180)
            chol = random.uniform(150, 300)
            sex = random.choice([0, 1])  # male=0, female=1

            # Normalize all numeric features to 0–1
            age_norm = (age - 20)/60
            bmi_norm = (bmi - 18.5)/11.5
            bp_norm = (bp - 90)/90
            chol_norm = (chol - 150)/150

            feat = [
                ecg_prob, exercise, diet, sleep,
                smoking, alcohol, age_norm, sex,
                bmi_norm, bp_norm, chol_norm
            ]
            self.features.append(torch.tensor(feat, dtype=torch.float32))

            # Simple risk: ECG dominates, others add minor contributions
            risk = min(max(
                0.5 * ecg_prob + 
                0.1 * (1 - exercise) + 
                0.1 * (1 - diet) + 
                0.1 * (1 - sleep) + 
                0.05 * smoking + 
                0.05 * alcohol + 
                0.1 * age_norm + 
                0.05 * bmi_norm + 
                0.05 * bp_norm + 
                0.05 * chol_norm,
                0), 1
            )

            self.targets.append(torch.tensor(risk, dtype=torch.float32))

        self.features = torch.stack(self.features).to(cfg.device)
        self.targets = torch.stack(self.targets).to(cfg.device)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.targets[idx]


In [72]:
def train_model(model, dataset, cfg=CFG):
    loader = DataLoader(dataset, batch_size=cfg.batch_size, shuffle=True)
    optimizer = torch.optim.Adam(model.parameters(), lr=CFG.lr)
    criterion = nn.MSELoss()

    for epoch in range(cfg.max_epochs):
        total_loss = 0
        model.train()
        for X, y in loader:
            optimizer.zero_grad()
            pred = model(X).squeeze()
            loss = criterion(pred, y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}/{cfg.max_epochs}, Loss: {total_loss/len(loader):.4f}")
    return model


def evaluate_model(model, dataset, cfg=CFG):
    model.eval()
    with torch.inference_mode():
        preds = model(dataset.features).squeeze()
        corr = torch.corrcoef(torch.stack([preds, dataset.targets]))[0,1]
    print(f"Correlation with target risk: {corr:.3f}")

In [73]:
dataset = SyntheticPatientDataset(num_samples=1000)
model = CardioRiskNet(input_size=dataset.features.shape[1]).to(CFG.device)

model = train_model(model, dataset)



Epoch 1/150, Loss: 0.0171
Epoch 2/150, Loss: 0.0155
Epoch 3/150, Loss: 0.0146
Epoch 4/150, Loss: 0.0137
Epoch 5/150, Loss: 0.0136
Epoch 6/150, Loss: 0.0134
Epoch 7/150, Loss: 0.0131
Epoch 8/150, Loss: 0.0124
Epoch 9/150, Loss: 0.0124
Epoch 10/150, Loss: 0.0121
Epoch 11/150, Loss: 0.0118
Epoch 12/150, Loss: 0.0115
Epoch 13/150, Loss: 0.0111
Epoch 14/150, Loss: 0.0108
Epoch 15/150, Loss: 0.0108
Epoch 16/150, Loss: 0.0100
Epoch 17/150, Loss: 0.0098
Epoch 18/150, Loss: 0.0092
Epoch 19/150, Loss: 0.0090
Epoch 20/150, Loss: 0.0085
Epoch 21/150, Loss: 0.0078
Epoch 22/150, Loss: 0.0073
Epoch 23/150, Loss: 0.0071
Epoch 24/150, Loss: 0.0068
Epoch 25/150, Loss: 0.0065
Epoch 26/150, Loss: 0.0065
Epoch 27/150, Loss: 0.0059
Epoch 28/150, Loss: 0.0053
Epoch 29/150, Loss: 0.0053
Epoch 30/150, Loss: 0.0052
Epoch 31/150, Loss: 0.0048
Epoch 32/150, Loss: 0.0048
Epoch 33/150, Loss: 0.0047
Epoch 34/150, Loss: 0.0042
Epoch 35/150, Loss: 0.0043
Epoch 36/150, Loss: 0.0040
Epoch 37/150, Loss: 0.0039
Epoch 38/1

In [74]:
evaluate_model(model, dataset)

Correlation with target risk: 0.985
Example patient predicted risk: 0.5655256509780884


In [75]:
def show_patient_metadata(features, true_risk, pred_risk):
    """
    features: 1D tensor of size 11
    true_risk: scalar tensor (0–1)
    pred_risk: scalar tensor (0–1)
    """

    (ecg_prob,
     exercise,
     diet,
     sleep,
     smoking,
     alcohol,
     age,
     sex,
     bmi,
     bp,
     chol) = features.tolist()

    print("\n================ Example Patient Metadata ================\n")

    print(f" ECG abnormality probability : {ecg_prob:.3f}")
    print(f" Exercise score (0–1)       : {exercise:.3f}")
    print(f" Diet quality (0–1)         : {diet:.3f}")
    print(f" Sleep quality (0–1)        : {sleep:.3f}")
    print(f" Smoking                    : {'Yes' if smoking > 0.5 else 'No'}")
    print(f" Alcohol use               : {'Yes' if alcohol > 0.5 else 'No'}")

    print(f" Age (normalized 0–1)       : {age:.3f}")
    print(f" Sex                        : {'Female' if sex > 0.5 else 'Male'}")
    print(f" BMI (normalized 0–1)       : {bmi:.3f}")
    print(f" Blood Pressure (0–1)       : {bp:.3f}")
    print(f" Cholesterol (0–1)          : {chol:.3f}")

    print("\n================ Risk Scores ================\n")
    print(f" Model Predicted Risk Score : {pred_risk:.3f}")
    print(f" True Risk Score            : {true_risk:.3f}")

    print("\n==================================================\n")


In [76]:
import random
idx = random.randint(0, 999)

features = dataset.features[idx]
true_risk = dataset.targets[idx]
pred_risk = model(features.unsqueeze(0)).item()

show_patient_metadata(features, true_risk, pred_risk)



================ Patient Metadata ================

 ECG abnormality probability : 0.240
 Exercise score (0–1)       : 0.525
 Diet quality (0–1)         : 0.957
 Sleep quality (0–1)        : 0.931
 Smoking                    : No
 Alcohol use               : No
 Age (normalized 0–1)       : 0.913
 Sex                        : Female
 BMI (normalized 0–1)       : 0.258
 Blood Pressure (0–1)       : 0.750
 Cholesterol (0–1)          : 0.154

================ Risk Scores ================

 Model Predicted Risk Score : 0.327
 True Risk Score            : 0.329




In [77]:
def analyze_dataset(dataset):
    """
    Computes and prints the average value of each input feature
    and the average target risk score.
    """
    features = dataset.features  # shape [N, 11]
    targets = dataset.targets    # shape [N]

    mean_feats = features.mean(dim=0)
    mean_target = targets.mean().item()

    (ecg_prob,
     exercise,
     diet,
     sleep,
     smoking,
     alcohol,
     age,
     sex,
     bmi,
     bp,
     chol) = mean_feats.tolist()

    print("\n================ Dataset Feature Averages ================\n")

    print(f" Avg ECG abnormality prob   : {ecg_prob:.3f}")
    print(f" Avg exercise score         : {exercise:.3f}")
    print(f" Avg diet score             : {diet:.3f}")
    print(f" Avg sleep score            : {sleep:.3f}")
    print(f" % Smokers                  : {smoking:.3f}")
    print(f" % Alcohol users            : {alcohol:.3f}")

    print(f" Avg age (normalized)       : {age:.3f}")
    print(f" % Female                   : {sex:.3f}")
    print(f" Avg BMI (normalized)       : {bmi:.3f}")
    print(f" Avg Blood Pressure (0-1)   : {bp:.3f}")
    print(f" Avg Cholesterol (0-1)      : {chol:.3f}")

    print("\n================ Label Averages ================\n")
    print(f" Avg Risk Score             : {mean_target:.3f}")

    print("\n===========================================================\n")


In [78]:
analyze_dataset(dataset)


================ Dataset Feature Averages ================

 Avg ECG abnormality prob   : 0.485
 Avg exercise score         : 0.506
 Avg diet score             : 0.506
 Avg sleep score            : 0.494
 % Smokers                  : 0.531
 % Alcohol users            : 0.509
 Avg age (normalized)       : 0.520
 % Female                   : 0.480
 Avg BMI (normalized)       : 0.519
 Avg Blood Pressure (0-1)   : 0.496
 Avg Cholesterol (0-1)      : 0.511

================ Label Averages ================

 Avg Risk Score             : 0.419




In [79]:
model.eval()

torch.save(model.state_dict(), "cardiorisknet_model.pt")